In [1]:
# Step 1: Upload the PDF
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving MTG_10009588.pdf to MTG_10009588.pdf


In [4]:
!pip install pdf2image pytesseract opencv-python-headless
!apt-get install -y poppler-utils
!pip install pymupdf
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import cv2
import numpy as np
import fitz  # PyMuPDF
import json
import tempfile
import os

# Step 2: Convert PDF to image using pdf2image
images = convert_from_path(filename)
image = images[0]  # Assume single-page document for now

# Step 3: Save image to disk for processing
image_path = "temp_image.jpg"
image.save(image_path, "JPEG")

# Step 4: Preprocess the image for better OCR
img_cv = cv2.imread(image_path)
gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Step 5: OCR and bounding box extraction
custom_config = r'--oem 3 --psm 6'
data = pytesseract.image_to_data(thresh, output_type=pytesseract.Output.DICT, config=custom_config)

# Step 6: Define key fields to extract
keywords = ['Borrower', 'Lender', 'Loan', 'Amount', 'Property', 'Address', 'Date', 'Recording', 'Mortgage', 'Deed']

extracted_info = []

# Step 7: Scan OCR output for keyword matches
for i in range(len(data['text'])):
    word = data['text'][i]
    if any(keyword.lower() in word.lower() for keyword in keywords):
        entry = {
            "text": word,
            "bbox": [data['left'][i], data['top'][i], data['left'][i] + data['width'][i], data['top'][i] + data['height'][i]]
        }
        extracted_info.append(entry)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 91.9 MB/s eta 0:00:00


In [5]:
# Step 8: Save to JSON
output_path = "scanned_mortgage_extraction.json"
with open(output_path, "w") as f:
    json.dump(extracted_info, f, indent=2)

# Step 9: Download the JSON file
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>